In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from fpdf import FPDF
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# Set parameters
num_days = 30  # Number of days of data
region = "Mangalore"
pollutants = ["PM2.5", "PM10", "NO2", "SO2", "O3", "CO"]

# Generate date range and random pollution data
start_date = datetime.now() - timedelta(days=num_days)
dates = [start_date + timedelta(days=i) for i in range(num_days)]

np.random.seed(0)  # For reproducibility
pollution_data = {
    "Date": [date.strftime("%Y-%m-%d") for date in dates],
    "Pollutant": [random.choice(pollutants) for _ in range(num_days)],
    "Region": [region] * num_days,
    "Pollutant Level (µg/m³)": np.random.uniform(10, 250, num_days)  # Random levels in realistic range
}

# Create DataFrame
df = pd.DataFrame(pollution_data)
report_summary = df.groupby("Pollutant")["Pollutant Level (µg/m³)"].describe()

# Define PDF report structure
class PDF(FPDF):
    def header(self):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, "Weather Pollution Report", ln=True, align="C")
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font("Arial", "I", 8)
        self.cell(0, 10, f"Page {self.page_no()}", 0, 0, "C")

    def chapter_title(self, title):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, f"{title}", ln=True)
        self.ln(5)

    def chapter_body(self, body):
        self.set_font("Arial", "", 12)
        self.multi_cell(0, 10, body)
        self.ln(5)

# Create PDF document
pdf = PDF()
pdf.add_page()

# Executive Summary
pdf.chapter_title("Executive Summary")
pdf.chapter_body("This report provides a summary of weather pollution data collected over a period of 30 days in Mangalore.")

# Table of Contents
pdf.chapter_title("Table of Contents")
pdf.chapter_body("1. Executive Summary\n2. Table of Contents\n3. Introduction\n4. Body\n5. Conclusion\n6. References\n7. Appendix")

# Introduction
pdf.chapter_title("Introduction")
pdf.chapter_body("This report analyzes air pollution data for Mangalore, capturing different pollutants' daily levels. The report's purpose is to provide insights on air quality and potential measures to address pollution.")

# Body (Main Report)
pdf.chapter_title("Body")
pdf.chapter_body("Below is the summary of pollution data:\n")
for pollutant, data in report_summary.iterrows():
    pdf.chapter_body(f"{pollutant}: Mean={data['mean']:.2f} µg/m³, Max={data['max']:.2f} µg/m³")

# Conclusion
pdf.chapter_title("Conclusion")
pdf.chapter_body("This analysis shows fluctuating pollution levels across various pollutants in Mangalore. Further measures can include pollution monitoring and alert systems to mitigate pollution impacts.")

# References
pdf.chapter_title("References")
pdf.chapter_body("Data generated for illustrative purposes. No external sources were used.")

# Appendix
pdf.chapter_title("Appendix")
pdf.chapter_body("Sample data collected over 30 days is shown below.")
for i, row in df.iterrows():
    pdf.chapter_body(f"Date: {row['Date']}, Pollutant: {row['Pollutant']}, Level: {row['Pollutant Level (µg/m³)']:.2f} µg/m³")

# Save PDF to Downloads folder
output_path = os.path.join(os.path.expanduser("~"), "Downloads", "Weather_Pollution_Report_Mangalore.pdf")
pdf.output(output_path)

print(f"Report saved as {output_path}")

# Email setup
sender_email = "prajnaatthaje@gmail.com"
receiver_email = "prajnapoojari14@gmail.com"
subject = "Weather Pollution Report for Mangalore - Past 30 Days"
body = """
Dear Authorizer,

Please find attached the weather pollution report for Mangalore for the past 30 days.
This report provides detailed insights into pollution levels and suggested precautions.

Best regards,
Prajnaatthaje Team
"""

# Function to send email
def send_email():
    # Create email message
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject

    # Attach body and PDF
    message.attach(MIMEText(body, "plain"))
    with open(output_path, "rb") as file:
        attach = MIMEApplication(file.read(), _subtype="pdf")
        attach.add_header("Content-Disposition", "attachment", filename="Weather_Pollution_Report_Mangalore.pdf")
        message.attach(attach)

    # Send the email using SMTP
    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            # Note: Replace 'your_app_password' with your Gmail app-specific password if needed
            server.login(sender_email, "cqpi gpyl gvgi ajpg")
            server.sendmail(sender_email, receiver_email, message.as_string())
            print("Email sent successfully.")
    except smtplib.SMTPException as e:
        print(f"An error occurred while sending the email: {e}")

# Call the function to send the email with the report attached
send_email()


Report saved as C:\Users\PRAJNA\Downloads\Weather_Pollution_Report_Mangalore.pdf
Email sent successfully.


In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
import numpy as np
import joblib

# Load DataFrame
df = pd.DataFrame(pollution_data)

# Feature Engineering
df['Date'] = pd.to_datetime(df['Date'])
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# Encode categorical features
label_enc_pollutant = LabelEncoder()
df['Pollutant'] = label_enc_pollutant.fit_transform(df['Pollutant'])

label_enc_region = LabelEncoder()
df['Region'] = label_enc_region.fit_transform(df['Region'])

# Prepare features and target variable
X = df[['Day', 'Month', 'Year', 'Pollutant', 'Region']]
y = df['Pollutant Level (µg/m³)']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train model
rf = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(rf, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)

# Best model
best_rf = grid_search.best_estimator_

# Evaluate on test data
y_pred = best_rf.predict(X_test)

# Metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R² Score: {r2:.2f}")

# Save model
joblib.dump(best_rf, "pollution_level_predictor.pkl")

# User input for prediction
print("\nEnter details to predict pollution level:")
day = int(input("Day (1-31): "))
month = int(input("Month (1-12): "))
year = int(input("Year: "))
pollutant = input(f"Pollutant ({', '.join(label_enc_pollutant.classes_)}): ")
region = input(f"Region ({', '.join(label_enc_region.classes_)}): ")

# Preprocess user input
pollutant_encoded = label_enc_pollutant.transform([pollutant])[0]
region_encoded = label_enc_region.transform([region])[0]

user_input = np.array([[day, month, year, pollutant_encoded, region_encoded]])
user_input_scaled = scaler.transform(user_input)

# Predict
predicted_level = best_rf.predict(user_input_scaled)
print(f"\nPredicted Pollution Level (µg/m³): {predicted_level[0]:.2f}")


Fitting 3 folds for each of 27 candidates, totalling 81 fits
MAE: 83.43
MSE: 7409.73
R² Score: -0.24

Enter details to predict pollution level:


Day (1-31):  14
Month (1-12):  05
Year:  2025
Pollutant (CO, NO2, O3, PM10, PM2.5, SO2):  CO
Region (Mangalore):  Mangalore



Predicted Pollution Level (µg/m³): 129.76


C:\Users\PRAJNA\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
